In [95]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [96]:
# 타이타닉 데이터 다운로드
!kaggle competitions download -c titanic

test.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)


In [97]:
## 데이터 분석 관련
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

## 데이터 시각화 관련
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid') # matplotlib의 스타일에 관련한 함
## 그래프 출력에 필요한 IPython 명령어
%matplotlib inline 

## Scikit-Learn의 다양한 머신러닝 모듈을 불러옵니다.
## 분류 알고리즘 중에서 선형회귀, 서포트벡터머신, 랜덤포레스트, K-최근접이웃 알고리즘을 사용해보려고 합니다.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [98]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [99]:
train_df.info()
print('-'*28)
test_df.info()
# train의 attribute는 12개 test의 attribute는 11개인 이유는 train은 Survied의 여부를 알고 있기 때문이다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
----------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-

###**여기서 주의 깊게 봐야할 부분은 다음과 같다.**
+ 각 데이터는 빈 부분이 있는가?
  + 빈 부분이 있다면, drop할 것인가 아니면 default값으로 채워 넣을 것인가.
  + cabin, Age, Embarked 세 항목에 주의
+ 데이터는 float64로 변환할 수 있는가. 
  + 아니라면 범주형 데이터로 만들 수 있는가.

In [100]:
# 이름과 티켓에서 가져올 수 있는 데이터는 없기 때문에 PassengerID와 이름, 티켓을 지운다. 
# 하지만 이 문제에서 결과물은 'PassengerId', 'Survived' 요소가 필요하므로 훈련데이터에서만 삭제한다.
train_df = train_df.drop(['PassengerId', 'Name', 'Ticket'], axis=1) # axis = 1은 열을 지운다.
test_df = test_df.drop(['Name', 'Ticket'], axis=1) # 결과물은 test에서 나온다. 즉, PassengerId를 지우면 안된다.

###**데이터 하나하나 처리하기**
이제 남은 데이터 종류는 다음과 같다.
1. Pclass
2. Sex
3. Age
4. SibSp
5. Parch
6. Fare
7. Cabin
8. Embarked

In [101]:
# 1. Pclass
# 서수형 데이터이다. 1등석, 2등석, 3등석과 같은 정보. 처음에 확인시에 데이터가 비어있지 않은 것을 확인할 수 있었다.
# 데이터에 대한 확인과 데이터를 변환해보도록 하겠다. 우선 각 unique한 value에 대한 카운팅은 value_counts() 메서드로 확인할 수 있다.
train_df['Pclass'].value_counts()

# 1, 2, 3은 정수이니, 그냥 실수로만 바꾸면 되지않을까 생각할 수 있다. 하지만 1, 2, 3 등급은 경우에 따라 다를 수 있지만 연속적인 정보가 아니며, 각 차이 또한 균등하지 않다.
# 그렇기에 범주형(카테고리) 데이터로 인식하고 인코딩해야한다.(비슷한 예시로 영화 별점 등이 있다.)
# 이 데이터는 범주형 데이터이므로 one-hot-encoding을 pd.get_dummies() 메서드로 인코딩한다.
pclass_train_dummies = pd.get_dummies(train_df['Pclass'])
pclass_test_dummies = pd.get_dummies(test_df['Pclass'])

train_df.drop(['Pclass'], axis=1, inplace=True)
test_df.drop(['Pclass'], axis=1, inplace=True)

train_df = train_df.join(pclass_train_dummies)
test_df = test_df.join(pclass_test_dummies)

In [102]:
train_df.head() # 원래는 columns의 이름을 설정하고, 넣어줘야하는데 실수로 넣지 않아 1, 2, 3 이라는 컬럼으로 데이터가 들어갔다.

,Survived,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,1,2,3
0,0,male,22.0,1,0,7.2500,NaN,S,0,0,1
1,1,female,38.0,1,0,71.2833,C85,C,1,0,0
2,1,female,26.0,0,0,7.9250,NaN,S,0,0,1
3,1,female,35.0,1,0,53.1000,C123,S,1,0,0
4,0,male,35.0,0,0,8.0500,NaN,S,0,0,1


In [103]:
# 2. Sex
# 성별이라는 뜻으로 남과 여로 나뉘므로 이 또한 one-hot-encoding을 진행
sex_train_dummies = pd.get_dummies(train_df['Sex'])
sex_test_dummies = pd.get_dummies(test_df['Sex'])

train_df.drop(['Sex'], axis=1, inplace=True)
test_df.drop(['Sex'], axis=1, inplace=True)

train_df = train_df.join(sex_train_dummies)
test_df = test_df.join(sex_test_dummies)

In [104]:
# 3. Age
# 나이는 연속형 데이터이므로, 큰 처리가 필요없다. (카테고리화를 하여 일부 알고리즘에 더 유용한 결과를 만들 수 있다.)
# 하지만 일부 NaN 데이터가 있으니 이를 채울 수 있는 방법에 대해서 생각해보자
# 1. 랜덤, 2. 평균값, 3. 중간값, 4. 데이터 버리기
# 여기서는 일단 평균값으로 채우도록 하겠다. 데이터의 통일성을 가지기 위해 train 데이터셋의 평균값으로 훈련, 테스트 데이터셋을 채우겠다.
train_df['Age'].fillna(train_df['Age'].mean(), inplace=True)
test_df['Age'].fillna(train_df['Age'].mean(), inplace=True)

In [105]:
# 4,5. SibSp & Panch 
# 형제 자매와 부모님은 가족으로 함께 처리할 수 있다. 하지만 굳이 바꿀필요는 없다.
# 6. Fare
# 탑승료이다. 신기하게 test 데이터셋에 1개의 데이터가 비어있다. 아마 디카프리오인듯 하다. 우선 빈 부분을 fillna 메서드로 채운다.
# 데이터 누락이 아닌 무단 탑승이라 생각하고 0으로 입력
test_df['Fare'].fillna(0, inplace=True)

In [106]:
# 7. Cabin
# 객실이라는 뜻이다. 대부분이 NaN이므로 버린다.
train_df = train_df.drop(['Cabin'],axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

In [107]:
# 8. Embarked
# 탑승 항구를 의미, 우선 데이터를 확인
train_df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [108]:
test_df['Embarked'].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [109]:
# S가 대다수이고 일부 데이터가 비어있는 것을 알 수 있다. 빈 부분은 S로 우선 채운다(.info로 확인했을 때 빈 부분이 있는 줄 몰랐다)
train_df["Embarked"].fillna('S', inplace=True)
test_df["Embarked"].fillna('S', inplace=True)

embarked_train_dummies = pd.get_dummies(train_df['Embarked'])
embarked_test_dummies = pd.get_dummies(test_df['Embarked'])

embarked_train_dummies.columns = ['S', 'C', 'Q']
embarked_test_dummies.columns = ['S', 'C', 'Q']

train_df.drop(['Embarked'], axis=1, inplace=True)
test_df.drop(['Embarked'], axis=1, inplace=True)

train_df = train_df.join(embarked_train_dummies)
test_df = test_df.join(embarked_test_dummies)

In [110]:
train_x = train_df.drop("Survived", axis=1)
train_y = train_df['Survived']
test_x = test_df.drop("PassengerId", axis=1).copy() # .copy()를 하는 이유?
# print(train_x.head())
# print(train_x.shape)
# print(train_y.shape)
# print(train_y[:10])

###**머신러닝 알고리즘 적용하기**

In [111]:
# Logistic Regression
logreg = LogisticRegression(max_iter=1000) # max_iter을 이용하여 오류해결
logreg.fit(train_x, train_y)

pred_y = logreg.predict(test_x)
logreg.score(train_x, train_y)

0.8047138047138047

In [112]:
# Support Vector Machines
svc = SVC()
svc.fit(train_x, train_y)

pred_y = svc.predict(test_x)
svc.score(train_x, train_y)

0.6868686868686869

In [113]:
# Random Forests
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(train_x, train_y)

pred_y = random_forest.predict(test_x)
random_forest.score(train_x, train_y)

0.9820426487093153

In [114]:
# K-Neigbor
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(train_x, train_y)

pred_y = knn.predict(test_x)
knn.score(train_x, train_y)

0.835016835016835

In [115]:
# Random Forests
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(train_x, train_y)
pred_y = random_forest.predict(test_x)
random_forest.score(train_x, train_y)

submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": pred_y
    })
submission.to_csv('titanic.csv', index=False)